# PERSIAPKAN DATA

Membersihkan Lingkungan Dari Data Sebelumnya

In [ ]:
import os, shutil

if os.path.exists('data.zip'): os.remove('data.zip')
if os.path.exists('data'): shutil.rmtree('data')

Download Data Yang Akan Dijadikan Dataset

In [ ]:
import os, gdown
from zipfile import ZipFile

gdown.download('https://drive.google.com/uc?id=1w4p9ugXJrUV-qmxt6UyJZLAzj0uefIvZ', 'data.zip', quiet=False)

if not os.path.exists('data'): os.makedirs("data")

with ZipFile('data.zip', 'r') as zObject:
	zObject.extractall('data')

# AUGMENTASI GAMBAR

Generator untuk membuat gambar menjadi lebih variatif dan menentukan besar data yang digunakan untuk validasi

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
    rescale = 1./255,
    rotation_range = 20,
    horizontal_flip = True,
    shear_range = 0.2,
    fill_mode = 'nearest',
    validation_split = 0.2
)

# DATASET

Ubah Data Menjadi Dataset Training & Validation

In [ ]:
lokasi = 'data'

training = datagen.flow_from_directory(
    lokasi,
    target_size = (50, 50),
    class_mode = 'binary',
    subset = 'training'
)

validation = datagen.flow_from_directory(
    lokasi,
    target_size = (50, 50),
    class_mode = 'binary',
    subset = 'validation'
)

# MODEL CNN

Build Model Sequential Untuk Klasifikasi 2 Gambar (biner) dengan 3 layer neuron dan 1 output

In [ ]:
from tensorflow import keras

model = keras.Sequential()

model.add(keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(50, 50, 3)))
model.add(keras.layers.MaxPool2D(2, 2))
model.add(keras.layers.Dropout(0.25))

model.add(keras.layers.Conv2D(64, (3, 3), activation='relu'))
model.add(keras.layers.MaxPool2D(2, 2))
model.add(keras.layers.Dropout(0.25))

model.add(keras.layers.Conv2D(128, (3, 3), activation='relu'))
model.add(keras.layers.MaxPool2D(2, 2))
model.add(keras.layers.Dropout(0.25))

model.add(keras.layers.Flatten())
model.add(keras.layers.BatchNormalization())

model.add(keras.layers.Dense(256, activation='relu'))
model.add(keras.layers.Dense(1, activation='sigmoid'))

model.summary()

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# LATIH

Model Yang Sudah Dibuat Dilatih Memakai Dataset Training Dan Validation

In [ ]:
import tensorflow as tf

accuracy_threshold = 85e-2
class my_callbacks(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs = None):
        if logs.get('val_accuracy') >= accuracy_threshold:
            self.model.stop_training = True
            
history = model.fit(training, steps_per_epoch=64, epochs=100, validation_data=validation, callbacks=[my_callbacks()])

Grafik Riwayat Pelatihan

In [ ]:
import matplotlib.pyplot as plt

accuracy = history.history['val_accuracy']
loss = history.history['val_loss']

plt.plot(accuracy, color='blue', label='Akurasi')
plt.plot(loss, color='red', label='Loss')
plt.xlabel('Epoch')
plt.ylabel('Nilai')
plt.title('Riwayat Pelatihan Model')
plt.legend()
plt.show()

Cek Akurasi Model

In [ ]:
eval_result = model.evaluate(validation)

accuracy = eval_result[1]
print(f'\nAkurasi Model: {accuracy * 100:.2f} %')

# Evaluasi

Menguji Model Dengan Berbagai Kemungkinan Gambar Di Luar Dataset

In [ ]:
import os, numpy as np, matplotlib.pyplot as plt
from keras.preprocessing import image

daftar_file = os.listdir('evaluasi')
daftar_file_urut = sorted(daftar_file, key=lambda x: int(os.path.splitext(x)[0]))

if os.path.exists('evaluasi'):
    for file in daftar_file_urut:
        nama_file = 'evaluasi/' + file 

        display = image.load_img(nama_file, target_size=None)
        plt.imshow(display)

        img = image.load_img(nama_file, target_size=(50, 50))

        img_array = image.img_to_array(img)
        img_array = np.expand_dims(img_array, axis=0)

        predictions = model.predict(img_array)

        if predictions[0][0] > 0.5:
            plt.xlabel(f'KURA - KURA', fontsize=15)
        else:
            plt.xlabel(f'BUAYA', fontsize=15)
        plt.show()
else :
    print('Silahkan buat folder evaluasi dan isi dengan gambar buaya dan kura kura acak terlebih dulu')
    exit